## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print(df)

       children  days_employed  dob_years education  education_id  \
0             1   -8437.673028         42    высшее             0   
1             1   -4024.803754         36   среднее             1   
2             0   -5623.422610         33   Среднее             1   
3             3   -4124.747207         32   среднее             1   
4             0  340266.072047         53   среднее             1   
...         ...            ...        ...       ...           ...   
21520         1   -4529.316663         43   среднее             1   
21521         0  343937.404131         67   среднее             1   
21522         1   -2113.346888         38   среднее             1   
21523         3   -3112.481705         38   среднее             1   
21524         2   -1984.507589         40   среднее             1   

          family_status  family_status_id gender income_type  debt  \
0       женат / замужем                 0      F   сотрудник     0   
1       женат / замужем        

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.columns

Index(['children', 'days_employed', 'dob_years', 'education', 'education_id',
       'family_status', 'family_status_id', 'gender', 'income_type', 'debt',
       'total_income', 'purpose'],
      dtype='object')

### Вывод

В таблице 21525строк и 12 стоблцов.В столбцах 'days_employed' и 'total_income' встречаются пропущенные значения. В столбце 'days_employed' указаны некорректные значения: отрицательные, либо слишком большие для того, чтобы быть стажем в днях.

### Шаг 2. Предобработка данных

### Обработка пропусков

Столбец стаж. Весь столбец состоит из отрицательных значений и встречаются слишком большие положительные значения. Такие значения замечены у пенсионеров. Проверим встречаются ли в других группах положительные данные.

In [4]:
df_big_employed = df[df['days_employed'] > 0]
df_big_employed['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

In [5]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [6]:
# Такие данные только у неработающих людей. Отберем строки с большим стажем
df[df['income_type'] == 'безработный']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3133,1,337524.466835,31,среднее,1,женат / замужем,0,M,безработный,1,59956.991984,покупка жилья для сдачи
14798,0,395302.838654,45,Высшее,0,гражданский брак,1,F,безработный,0,202722.511368,ремонт жилью


In [7]:
df[(df['income_type'] == 'пенсионер') & (df['dob_years'] < 50)].sort_values('days_employed', ascending=False)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14514,0,400992.375704,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,87215.410371,заняться высшим образованием
11005,0,400954.247845,49,среднее,1,гражданский брак,1,F,пенсионер,0,90403.089103,сыграть свадьбу
4692,0,400556.218389,46,среднее,1,женат / замужем,0,F,пенсионер,0,99148.330938,покупка жилья для семьи
10288,0,400553.798436,42,начальное,3,гражданский брак,1,F,пенсионер,1,136297.922676,свадьба
15897,0,400541.408715,47,высшее,0,женат / замужем,0,M,пенсионер,0,208201.202793,дополнительное образование
...,...,...,...,...,...,...,...,...,...,...,...,...
17696,1,NaN,45,начальное,3,женат / замужем,0,F,пенсионер,0,NaN,операции с недвижимостью
18175,2,NaN,31,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба
18255,0,NaN,38,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,профильное образование
19607,0,NaN,41,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба


In [ ]:
# Т.к. единица измерения стажа для неработающих точно не дни, посчитаем сколько лет получится, 
# если предположить, что это часы или минуты. 
employed_max_anomaly_h = df['days_employed'].max() / (365 * 24)
employed_max_anomaly_m = df['days_employed'].max() / (365 * 24 * 60)
print('Если значения в стаже указаны в часах, то максимальный стаж среди неработающих {:.1f} лет'.format(employed_max_anomaly_h))
print('Если значения в стаже указаны в минутах, то максимальный стаж среди неработающих {:.1f} лет'.format(employed_max_anomaly_m))

In [9]:
employed_max_year = df['days_employed'].min() / (-365)
round(employed_max_year)

50

Среди минусовых значений максимальный стаж в днях равен пятидесяти годам. Похоже на правду.

In [10]:
df['days_employed'].isnull().sum()

2174

In [11]:
# В столбцах со стажем и доходом было одинаковое количество пропусков - 2174. Это 10% от всех строк - слишком много, 
# чтобы их отбросить.Заменим отсутствующие значения в стаже и доходах на медианные значение для каждого вида
# источника доходов.
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['total_income'].transform('median'))

In [12]:
# В столбце с возрастом есть нулевые значения
df[df['dob_years'] == 0].head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль


In [13]:
# Поменяем нули на медианные значения возраста в каждой группе источника доходов
df.loc[df['dob_years'] == 0, 'dob_years'] = df.groupby('income_type')['dob_years'].transform('median')
df[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [ ]:
# Заменим "большой" стаж в столбце 'years_employed' на разницу между возрастом и совершеннолетием
df.loc[(df['income_type'] == 'пенсионер') | (df['income_type'] == 'безработный'), 'days_employed'] = df['dob_years'] * 365 - 18 * 365
df

In [15]:
# Переведем отрицательные значения в положительные
df['days_employed'] = df['days_employed'].abs()
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33.0,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,12775.000000,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


### Вывод

У всех неработающих категорий (пенсионеров и безработных) стаж отражен положительным числом. Эти значения слишком большие, чтобы быть стажем в днях. На часы тоже не похоже, т.к. в этом случае, например, у одного из безработных, стаж получается больше возраста, а второй начал работать с рождения. С пенсионерами также. В других категориях такой особенности не замечено. Скорее всего эти значения не стаж, а время в минутах, которое прошло с момента, как человек перестал работать, т.к. в этом случае максимальное значение в столбце получается меньше года. 
Отрицательные значения видимо связаны с техническими причинами. Учитывая то, что это проблема целого столбца, а не единичных значений, возможно при выгрузке каким-то образом добавился минус.

### Замена типа данных

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


In [17]:
# Для удобства переведем данные типа float64 в целочисленные
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df['dob_years'] = df['dob_years'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [18]:
# Проверим какие значения содержат остальные столбцы
# Детей 20 или -1 - Доля таких данных 0,5%

df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

В столбце 'children' содержатся значения сильно выделяющиеся среди остальных: "20" и "-1". Очевидно, что это ошибка. Похоже, что при внесении данных случано задели "0", а под минусом предполагалось тире. Таких данных всего 0,5%, поэтому заменим их на "2" и "1", которые входят в тройку самых распространенных.

In [19]:
df['children'] = df['children'].replace([-1, 20], [1, 2])
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [20]:
# Кто такой XNA?
df['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [21]:
df.groupby('gender')['total_income'].mean()

gender
F      153047.957362
M      189005.150796
XNA    203905.000000
Name: total_income, dtype: float64

In [22]:
df_xna = df[df['gender'] == 'XNA']
df_xna

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


В столбце 'gender' есть человек с третьим полом. Думаю, это не ошибка, а человек с медицинсками проблемами (гермафродит), или представитель тех кто не считает себя ни мужчиной ни женщиной - в некоторых странах сейчас это распространено. Исходя из среднего дохода, добавим его к мужчинам.

In [23]:
df['gender'] = df['gender'].replace('XNA', 'M')
df['gender'].value_counts()

F    14236
M     7289
Name: gender, dtype: int64

### Вывод

Столбцы с числовыми данными было тяжело воспринимать из-за большого количества знаков после запятой. К этим столбцам был применен метод astype() и получены целые числа. В таблице не осталось пропусков и аномальных числовых значений, данные имеют тип int64
или object.

### Обработка дубликатов

In [24]:
# Приводим слова в таблице к одинаковому написанию слов (строчными буквами)
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['purpose'] = df['purpose'].str.lower()

df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


In [25]:
df.duplicated().sum()

71

In [26]:
# Удалим обнаруженные дубликаты
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

In [27]:
# Проверим столбцы со строковыми значениями на наличие повторов
df['education'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [28]:
df['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [29]:
df['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [30]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

### Вывод

После перевода всех слов к одному формату, был обнаружен 71 дубликат. Возможно, это связано с выгрузкой/загрузкой данных в разные периоды времени, по мере накопления, например, раз в месяц/год и значения крайней даты дублировались. А то, что до перевода строк к одному формату дубликатов было меньше, наводит на мысль, что данные могли быть взяты из разных источников, либо получены с помощью разных инструментов. Например, если один человек обращалася за кредитом в разные банки. Повторные строки были удалены с помощью drop_duplicates(), но в столбце с целью кредита осталось много повторов, связанных с разными формулировками целей. 

### Лемматизация

In [31]:
# Лемматизируем столбец 'purpose', чтобы получить список лемм в каждой строке
from pymystem3 import Mystem
m = Mystem()
df['lemmas'] = df.purpose.map(m.lemmatize)
df['lemmas'].value_counts()

[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

In [32]:
# Выведем список всех встречающихся слов с их количеством
from collections import Counter
Counter(m.lemmatize(' '.join(df.purpose)))

Counter({'покупка': 5897,
         ' ': 55023,
         'жилье': 4460,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

### Вывод

В результате лемматизации все цели можно разделить на пять категорий: свадьба, ремонт, автомобиль, образование и недвижимость.

### Категоризация данных

In [33]:
# Применим к строкам таблицы функцию для присваивания категорий в зависимости от целей кредита
# и посчитаем количество значений для каждой из них
def purpose_category(row):
    if 'свадьба' in row['lemmas']:
        return 'свадьба'
    if 'ремонт' in row['lemmas']:
        return 'ремонт'
    if 'автомобиль' in row['lemmas']:
        return 'автомобиль'
    if 'образование' in row['lemmas']:
        return 'образование'
    else:
        return 'недвижимость'
        
df.apply(purpose_category, axis = 1).value_counts()


недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
dtype: int64

In [34]:
# Сохраним категории в отдельный столбец
            
df['loan_category'] = df.apply(purpose_category, axis = 1)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,loan_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба


In [35]:
# Так как заемщиков с 4-5 детьми слишком мало, выделим категории в зависимости от количества детей:
# 0 детей - категория 'без детей', от 1 до 2 - 'малодетные', от 3 детей и больше - 'многодетные'
def children_category(children):
    if children == 0:
        return 'без детей'
    if children >= 3:
        return 'многотеные'
    else:
        return 'малодетные'
df['children_category'] = df['children'].apply(children_category)
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,loan_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,малодетные
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,малодетные
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,без детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,многотеные
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,без детей


In [36]:
# Разделим данные о доходах на равные интервалы и сохраним в отдельный столбец
df['income_category'] = pd.qcut(df['total_income'], 7)
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,loan_category,children_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,малодетные,"(242019.571, 2265604.0]"
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,малодетные,"(87295.429, 113605.571]"
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,без детей,"(137468.0, 155307.571]"
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,многотеные,"(242019.571, 2265604.0]"
4,0,12775,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,без детей,"(155307.571, 184477.286]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21449,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,малодетные,"(184477.286, 242019.571]"
21450,0,17885,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем,"[сделка, , с, , автомобиль, \n]",автомобиль,без детей,"(155307.571, 184477.286]"
21451,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость,"[недвижимость, \n]",недвижимость,малодетные,"(87295.429, 113605.571]"
21452,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля,"[на, , покупка, , свой, , автомобиль, \n]",автомобиль,многотеные,"(242019.571, 2265604.0]"


### Вывод

Информация о целях кредита была разделена на пять категорий, чаще всего люди обращаются в банк за кредитом на покупку/строительство недвижимости. Так как семей с 4-5 детьми слишком мало, количество детей разделили на три категории: 'без детей', 'малодетные', 'многодетные'. Доходы разделили на семь равных интервалов, чтобы лучше прослеживалась зависимость между доходами и возвратом кредита.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [49]:
df.groupby('children_category')[['debt']].mean()

,debt
children_category,
без детей,0.075438
малодетные,0.092654
многотеные,0.081579


In [38]:
df.pivot_table('debt', ['gender'], 'children_category')

children_category,без детей,малодетные,многотеные
gender,,,
F,0.062185,0.086819,0.077922
M,0.103041,0.102734,0.087248


In [39]:
df.pivot_table(index='gender', columns='children_category', aggfunc={'debt':sum})

debt                      
children_category без детей малодетные многотеные
gender                                           
F                       592        384         18
M                       471        263         13

### Вывод

В целом, люди без детей возвращают кредиты в срок чаще тех у кого есть дети. Хотя, если посмотреть на этот показатель отдельно по мужчинам и женщинам, то у мужчин от наличия детей он почти не меняется. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
df.pivot_table(index='family_status', values='debt', aggfunc=['count', 'sum','mean']).sort_values(('mean','debt'))

,count,sum,mean
,debt,debt,debt
family_status,,,
вдовец / вдова,959,63,0.065693
в разводе,1195,85,0.071130
женат / замужем,12339,931,0.075452
гражданский брак,4151,388,0.093471
не женат / не замужем,2810,274,0.097509


In [41]:
df.pivot_table('debt', index='family_status', columns='children_category')

children_category,без детей,малодетные,многотеные
family_status,,,
в разводе,0.070153,0.072682,0.083333
вдовец / вдова,0.062574,0.095238,0.000000
гражданский брак,0.083883,0.111439,0.121212
женат / замужем,0.069095,0.086132,0.070175
не женат / не замужем,0.092838,0.115242,0.200000


In [42]:
df.pivot_table('debt', index='family_status', columns='loan_category')

loan_category,автомобиль,недвижимость,образование,ремонт,свадьба
family_status,,,,,
в разводе,0.074733,0.065934,0.071429,0.128205,NaN
вдовец / вдова,0.091743,0.050980,0.075377,0.062500,NaN
гражданский брак,0.117512,0.094218,0.148515,0.054545,0.080034
женат / замужем,0.083699,0.070544,0.083237,0.048969,NaN
не женат / не замужем,0.128728,0.082502,0.107452,0.064516,NaN


### Вывод

Люди, которые не состоят в официальном браке, с семейным статусом 'гражданский брак' и 'не женат / не замужем' хуже отдают кредиты и чем больше у них детей, тем доля невозвратов больше. Видимо официальный брак - это все-таки не просто штамп в паспорте, а показатель готовности человека брать на себя ответсвенность и обдуманно принимать решения.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [43]:
df.groupby('income_category')[['debt']].mean()

,debt
income_category,
"(20666.999, 87295.429]",0.076020
"(87295.429, 113605.571]",0.085481
"(113605.571, 137468.0]",0.086758
"(137468.0, 155307.571]",0.086843
"(155307.571, 184477.286]",0.084176
"(184477.286, 242019.571]",0.079935
"(242019.571, 2265604.0]",0.068842


In [44]:
df.groupby(['income_type', 'income_category'])['debt'].aggregate('mean').unstack()

income_category,"(20666.999, 87295.429]","(87295.429, 113605.571]","(113605.571, 137468.0]","(137468.0, 155307.571]","(155307.571, 184477.286]","(184477.286, 242019.571]","(242019.571, 2265604.0]"
income_type,,,,,,,
безработный,1.000000,NaN,NaN,NaN,NaN,0.000000,NaN
в декрете,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
госслужащий,0.088542,0.061321,0.063953,0.046512,0.037838,0.066351,0.052863
компаньон,0.090909,0.074954,0.079646,0.084158,0.075652,0.072607,0.061962
пенсионер,0.046488,0.050633,0.067661,0.065972,0.054054,0.061538,0.055016
предприниматель,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
сотрудник,0.088348,0.105200,0.103638,0.095125,0.105882,0.090734,0.080586
студент,NaN,0.000000,NaN,NaN,NaN,NaN,NaN


### Вывод

Из таблиицы с долей должников в зависимости от доходов видно, что лучше всего отдают кредиты люди с очень низким и самым высоким доходом. С большими доходами в принципе понятно: если есть деньги, почему бы не следовать графику платежей и "делать" себе хорошую кредитную историю? А вот в меньшей доле просрочек в категории с низкими доходами, причина все-таки не в маленьких доходах, а в категориях его получения. Из второй таблицы видно, что независимо от суммы дохода, пенсионеры отдают кредиты лучше. Они обычно умеют экономить и не любят копить долги. Также, меньшей долей долгов по типу дохода выделяются госслужащие, которые наверняка имеют  льготы на покупку жилья/ипотеку. 

Как разные цели кредита влияют на его возврат в срок?

In [45]:
df.groupby('loan_category')[['debt']].mean() 

,debt
loan_category,
автомобиль,0.093590
недвижимость,0.073207
образование,0.092200
ремонт,0.057661
свадьба,0.080034


In [46]:
age = pd.cut(df['dob_years'], [0, 30, 40, 50, 60, 75])
df.pivot_table('debt', [age], 'loan_category')

loan_category,автомобиль,недвижимость,образование,ремонт,свадьба
dob_years,,,,,
"(0, 30]",0.122449,0.098938,0.129346,0.083969,0.096939
"(30, 40]",0.118243,0.083939,0.104869,0.076433,0.088957
"(40, 50]",0.084809,0.071174,0.086134,0.013605,0.085868
"(50, 60]",0.068256,0.054884,0.072261,0.054054,0.064516
"(60, 75]",0.056818,0.041667,0.050481,0.065574,0.044444


In [47]:
df.pivot_table('debt', 'education', 'loan_category')

loan_category,автомобиль,недвижимость,образование,ремонт,свадьба
education,,,,,
высшее,0.064672,0.050000,0.058887,0.018750,0.045000
начальное,0.109091,0.117647,0.090909,0.000000,0.142857
неоконченное высшее,0.104575,0.092437,0.091603,0.043478,0.075000
среднее,0.102547,0.079599,0.103043,0.074519,0.092203
ученая степень,NaN,0.000000,0.000000,NaN,NaN


### Вывод

Кредиты, взятые на ремонт, возвращают в срок чаще всего. Следом идет - недвижимость. Скорее всего это связано с тем, что решения о покупке недвижимости или ремонте принимаются более осознанно и не могут быть результатом мимолетных желаний. Также прослеживаются зависимости между возвратом кредита в срок и наличем высшего образования и чем старше человек, тем ответственне.

### Шаг 4. Общий вывод

Лучше всего отдают кредиты люди когда-либо состоящие в браке, без детей, имеющие доход выше 240000 и обращающиеся за кредитом с целью ремонта, либо приобретения жилья. Кроме того, отмечена особенность, что если человек относится к категориям пенсионер или госслужащий, имеет высшее образование и к тому же женского пола, то шансы возврата кредита в срок повышаются.